\begin{equation}
    k_B = J = 1
\end{equation}

In [1]:
using Pkg
Pkg.activate("./Packages/Project.toml");

  Activating project at `~/Repos/Heisenberg-Model-3D/Packages`


In [2]:
using LinearAlgebra
using Random

In [3]:
function RandomSpin()
    """Creates a random unit vector from an homogenous distribution
    on the unit sphere"""
    θ = acos(1-rand(0:1e-6:2))
    ϕ = rand(0:1e-6:2π)
    return [sin(θ)*cos(ϕ), sin(θ)*sin(ϕ), cos(θ)]
end

function AcceptanceProbability(Spin1, Spin2, n, β)
    """Computes the probability of adding Spin2 - neighbour of
    Spin1 - to the cluster. It uses the temperature β and the 
    vector of reflection n"""
    return 1 - exp(min(0,-2β*transpose(Spin1)*n*transpose(Spin2)*n))
end

function ReflectSpin(S,n,state)
    """Reflectes the spin state[S] around the plane defined by the
    normal vector n"""
    state[S] -=  2(transpose(state[S])*n)*n
end

function GetNeighbours(L,L2,L3)
    """Gets the coordinates of the 6 neighbours of every spin in the state. It uses 
    periodic boundery conditions in a 3D lattice of length L (area L2, volume L3). 
    The L3 × 6 matriz it returns contains the neighbours as folows: 
    [Up-Down-Left-Right-Front-Back]"""
    Neighbours = zeros(Int16,L3,6)
    for x in 1:L3
        Neighbours[x,:] = [L2*((x-1)÷L2)+(x-1-L+L2)%L2+1,
            L2*((x-1)÷L2)+(x-1+L)%L2+1,
            L*((x-1)÷L)+(x+L-2)%L+1,
            L*((x-1)÷L)+(x)%L+1,
            (x-L2+L3-1)%L3+1,
            (x+L2-1)%L3+1]
    end
    return Neighbours
end

function Grow_Reflect(S, Cluster, n, β, L, L2, L3, state, neighbours)
    """Checks every neighbour of spin state[S] and adds them to the Cluster given
        a certain probability (refair to AcceptanceProbability). Finally, it 
        reflects the spin around the normal vector n (refair to ReflectSpin)"""
    #Checks neighbours
    for Sn in neighbours[S,:]
        if Sn ∉ Cluster && rand(0:1e-15:1) < AcceptanceProbability(state[S],state[Sn],n,β)
            #Adds neighbour to Cluster
            push!(Cluster,Sn)
            #Checks neighbours of neighbour
            Grow_Reflect(Sn, Cluster, n, β, L, L2, L3, state, neighbours)
        end
    end
    #Reflects the spin
    ReflectSpin(S,n, state)
end

function NewState(state, L, L2, L3, β, neighbours)
    """Chooses a randon spin from the state with lattice sice L, and also a
    random normal vector. From there it builds the cluster using Wolff algorithm
    (see Grow_Reflect)"""
    n = RandomSpin()     #Initial random Normal Vector
    S0 = rand(1:1:L3)    #Initial random Spin
    Cluster = [S0]       #Stores the indexes of the spins added to the cluster
    Grow_Reflect(S0, Cluster, n, β, L, L2, L3, state, neighbours) #Builds cluster and flips for new state
    return length(Cluster)
end;

In [4]:
function Magnetization(state)
    """Returns the magnetization as the norm of the sum of all spins in the grid"""
    return norm(sum(state))
end

function Energy(state,L3,neighbours)
    """Returns the energy of a state"""
    energy = 0.0
    #Goes through every spin in the lattice
    for ii in 1:L3  
        Sp = [0,0,0]
        #Only checks 3 out the 6 neighbourds to avoid double counting of bounderies
        for Sn in neighbours[ii,[2,4,6]] 
           Sp += state[Sn]
        end
        energy -= dot(Sp,state[ii])
    end
    return energy
end;

In [5]:
function Proceed(state, β, n_samples, L, L2, L3, neighbours,τ)
    """For a single value of β in a lattice of size L, creates a sequence of 
    states and meassures the observables for the de-correlated configurations.
    Then it prints the desired quantities."""
    Mmean,M2mean,M4mean,Emean,E2mean,p = zeros(6)
    progress = 0      #Samples taken
    while progress < n_samples
        M = Magnetization(state); E = Energy(state,L3,neighbours)
        Mmean += M; M2mean += M*M; M4mean += M^4 
        Emean += E; E2mean += E*E
        t = 0         #Current virtual time in MCSS
        while t < τ 
            mcss = 0  #Number of spins flipped
            while mcss < L3 
                mcss += NewState(state, L, L2, L3, β, neighbours) 
            end 
        t += 1 
        end
        p += NewState(state, L, L2, L3, β, neighbours) #Additional step just to get the cluster size
        progress += 1 
    end
    Mmean /= n_samples; M2mean /= n_samples; M4mean /= n_samples
    Emean /= n_samples; E2mean /= n_samples; p /= n_samples
    χ  = β*(M2mean-Mmean*Mmean)
    Cv = β*β*(E2mean-Mmean*Mmean)
    U  = 1. - 1. /3. *(M4mean/(M2mean*M2mean))
    print(β," ",Mmean," ",M2mean," ",M4mean," ",Emean," ",E2mean," ",p," ",χ," ",Cv," ",U,"\n")
end;

In [6]:
"""----- MAIN -----"""
L = 10  #Lattice size
L2 = L*L   #Lattice Area
L3 = L*L2  #Lattice Volume
n_samples = 1000
τ = 10
#Random.seed!(51)
neighbours = GetNeighbours(L,L2,L3)  #Matrix with the indexes of the 6 adjacent spins of each in the lattice
Temperatures = [kk for kk in 0.7:.1:2.1 ]
βs = 1 ./ Temperatures;

In [7]:
for kk in βs                         #Loop over all tempetures
    state = [RandomSpin() for i in 1:L3] #The 3D lattice is store as a 1D-Array
    Proceed(state, kk, n_samples, L, L2, L3, neighbours,τ)
end

1.4285714285714286 811.5385837103754 659314.1845915567 4.3535446993240735e11 -2241.820172281932 5.031409046363582e6 504.981 1027.5882013067271 8.924110558189673e6 0.666161120145892
1.25 776.9775739970197 604377.8044486349 3.6594692629175903e11 -2117.8935361765216 4.490540316019177e6 478.94 854.5674429257633 6.07319713363263e6 0.6660512396222213
1.1111111111111112 740.5961972675726 549162.0380196997 3.022593810190027e11 -1988.9950456602066 3.9611816437109965e6 414.208 754.7895694560268 4.213208538646676e6 0.6659145838587368
1.0 698.8989753660751 489167.57012599445 2.4003110772360657e11 -1850.0877501132693 3.4276082930445806e6 365.113 707.7923582448275 2.939148515276831e6 0.6656271860806002
0.9090909090909091 650.0836459882772 423390.3793114398 1.800801733282495e11 -1703.1270703023638 2.9054080823441534e6 319.038 710.5750272982351 2.05190027732458e6 0.6651404613018921
0.8333333333333334 592.2551936587544 351686.87311681383 1.246156164807812e11 -1535.712382248051 2.3633047657591e6 262.103